## Search Engine With Tools and Agent


In [1]:
## Arxiv and Wikipedia Query Tools

from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper ,WikipediaAPIWrapper

In [2]:
#Used the InBuilt Tool of WikiPedia
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results = 1 , doc_content_chars_max = 250)
wiki = WikipediaQueryRun(api_wrapper = api_wrapper_wiki)
wiki.name

'wikipedia'

In [3]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results = 1 , doc_content_chars_max = 250)
arxiv  =  ArxivQueryRun(api_wrapper = api_wrapper_arxiv)

In [4]:
tools = [wiki , arxiv]


In [5]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_huggingface import HuggingFaceEmbeddings

os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

c:\Users\fk748\OneDrive\Desktop\SearchEngine_Tools&Agents\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
## Custom Tools 
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [7]:
#Loading LangSmith Docs 
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectordb = FAISS.from_documents(documents, embeddings)

In [8]:
retreiver = vectordb.as_retriever()
retreiver

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000187E0A971C0>, search_kwargs={})

In [9]:
import langchain
print(langchain.__version__)

1.2.7


In [10]:
# converting retreiver into a tool
from langchain_core.tools import create_retriever_tool
retreiver_tool = create_retriever_tool(retreiver , "langsmith-search" ,"Search any information about LangSmith from LangChain Docs")

retreiver_tool

StructuredTool(name='langsmith-search', description='Search any information about LangSmith from LangChain Docs', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=<function create_retriever_tool.<locals>.func at 0x00000187E0AA63B0>, coroutine=<function create_retriever_tool.<locals>.afunc at 0x00000187E0AC1990>)

In [11]:
tools = [arxiv , wiki , retreiver_tool]

In [12]:
tools

[ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\fk748\\OneDrive\\Desktop\\SearchEngine_Tools&Agents\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 StructuredTool(name='langsmith-search', description='Search any information about LangSmith from LangChain Docs', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=<function create_retriever_tool.<locals>.func at 0x00000187E0AA63B0>, coroutine=<function create_retriever_tool.<locals>.afunc at 0x00000187E0AC1990>)]

In [13]:
## Run all these tools and Agents with LLM Models
from langchain_groq import ChatGroq
## Tools , LLM -> Executer
groq_api_key = os.getenv('GROQ_API_KEY')
llm = ChatGroq(groq_api_key= groq_api_key ,model_name="llama-3.1-8b-instant")

In [19]:
## Prompt Template  => Using Hub => Provides in built prompts and templates

# from langchainhub import hub
# prompt = hub.pull('hwchase17/openai-functions-agent')
# prompt.message
# 
# Hub Not Working 
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant."),
    
    # Chat history
    MessagesPlaceholder(variable_name="chat_history"),
    
    # User input
    ("human", "{input}"),
    
    # Scratchpad for intermediate reasoning (used by agents)
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')]

In [26]:
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt)
agent

from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent  = agent , tools = tools , verbose=True)

agent_executor.invoke({"input" : "Tell me About LangSmith"})

ImportError: cannot import name 'create_openai_tools_agent' from 'langchain.agents' (c:\Users\fk748\OneDrive\Desktop\SearchEngine_Tools&Agents\venv\lib\site-packages\langchain\agents\__init__.py)